In [2]:
import pandas as pd

#import spark
from pyspark.sql import SparkSession
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 ass2 BNPL group 28")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

22/10/09 15:17:47 WARN Utils: Your hostname, Rudyletsgo resolves to a loopback address: 127.0.1.1; using 172.25.252.162 instead (on interface eth0)
22/10/09 15:17:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/10/09 15:17:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
## read the merchant data
merchant_info = spark.read.options(header = True).csv('../data/curated/merchant_info.csv')
merchant_tents = merchant_info.where(merchant_info.field == "tent and awning shops")
merchant_jewl = merchant_info.where(merchant_info.field == "jewelry, watch, clock, and silverware shops")
merchant_shoe = merchant_info.where(merchant_info.field == "shoe shops")

# Transform to pandas dataframe
merchant_info = merchant_info.toPandas()
merchant_tents = merchant_tents.toPandas()
merchant_jewl = merchant_jewl.toPandas()
merchant_shoe = merchant_shoe.toPandas()

## convert dataset columns into fesible types
merchant_info[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']] \
    = merchant_info[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']].apply(pd.to_numeric)

merchant_tents[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']] \
    = merchant_tents[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']].apply(pd.to_numeric)

merchant_jewl[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']] \
    = merchant_jewl[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']].apply(pd.to_numeric)

merchant_shoe[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']] \
    = merchant_shoe[['transaction_count', 'take_rate', 'total_revenue', 'mean_consumer_income', 'fraud_count', 'main_business_area_popu']].apply(pd.to_numeric)

#merchant_tents = merchant_info.loc[merchant_info['field'] == "tent and awning shops"].copy()
#merchant_jewl = merchant_info.loc[merchant_info['field']  == "jewelry, watch, clock, and silverware shops"].copy()
#merchant_shoe = merchant_info.loc[merchant_info['field'] == "shoe shops"].copy()

In [4]:
list(merchant_info.keys())

['merchant_abn',
 'transaction_count',
 'field',
 'take_rate',
 'revenue_level',
 'total_revenue',
 'mean_consumer_income',
 'fraud_count',
 'main_business_area_popu']

### Create score criteria(standard version)

In [5]:
## Manually set the score criteria, could be changed later.
score_criteria = { \
    'transaction_count': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
        'take_rate': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
            'revenue_level': {'a':0, 'b': 1, 'c': 2, 'd': 3, 'e': 4}, \
                'total_revenue': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
                    'mean_consumer_income': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
                        'fraud_count': {'a':0, 'b': 0, 'c': 0, 'd': 0, 'e': 0}, \
                            'main_business_area_popu': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}}

In [6]:
## store all numeric features into list for use in both Algorithms
featrue_name = list(merchant_info.keys())
featrue_name.remove('merchant_abn')
featrue_name.remove('field')

In [7]:
merchant_info

,merchant_abn,transaction_count,field,take_rate,revenue_level,total_revenue,mean_consumer_income,fraud_count,main_business_area_popu
0,16903677169,51,shoe shops,4.62,b,18506.39,795.935292,0,211425
1,21359184622,18011,motor vehicle supplies and new parts,3.60,b,5659723.28,832.634162,3,130894
2,21782713770,294,tent and awning shops,3.31,b,336040.60,796.557143,0,194701
3,23692710924,3248,"florists supplies, nursery stock, and flowers",5.61,a,969325.83,837.889286,0,116502
4,25499263184,64,"books, periodicals, and newspapers",6.14,a,32355.13,808.223438,0,44114
...,...,...,...,...,...,...,...,...,...
4021,43833568675,318,telecom,4.69,b,814552.89,791.296540,1,370220
4022,56923368251,146,"watch, clock, and jewelry repair shops",6.14,a,25387.87,768.678083,0,162711
4023,59190126409,174,"florists supplies, nursery stock, and flowers",6.87,a,60788.54,786.111494,0,198187
4024,83177825742,9206,"gift, card, novelty, and souvenir shops",5.02,b,1012564.70,833.715620,5,53266


In [8]:
score_criteria

{'transaction_count': {'a': 4, 'b': 3, 'c': 2, 'd': 1, 'e': 0},
 'take_rate': {'a': 4, 'b': 3, 'c': 2, 'd': 1, 'e': 0},
 'revenue_level': {'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4},
 'total_revenue': {'a': 4, 'b': 3, 'c': 2, 'd': 1, 'e': 0},
 'mean_consumer_income': {'a': 4, 'b': 3, 'c': 2, 'd': 1, 'e': 0},
 'fraud_count': {'a': 0, 'b': 0, 'c': 0, 'd': 0, 'e': 0},
 'main_business_area_popu': {'a': 4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}}

## Mark Algorithm

In [9]:
## Function used to assign continuous features into level (a,b,c,d,e) based on current merchant infomation
## merchant_info (a pd.DataFrame) is the current merchant information that are considered in our ranking system
## feature (a string) is the numeric feature we are working on
## score_criteria (a dict of dict) is the score criteria we are using for marking different features.

def mark_algorithm(merchant_info, feature, score_criteria):

    label_list = []
    score_list = []

    ## in each run of the ranking system, each numeric column is allocated with a level in (a,b,c,d,e)
    ## according to the overall quantile statistic of the feature column we are working on
    ab = merchant_info[feature].quantile(0.8)
    bc = merchant_info[feature].quantile(0.6)
    cd = merchant_info[feature].quantile(0.4)
    de = merchant_info[feature].quantile(0.2)
    
    for i in range(len(merchant_info)):
        if merchant_info.loc[i, feature] >= ab:
            label_list.append('a')
        elif (merchant_info.loc[i, feature] >= bc) & (merchant_info.loc[i, feature] < ab):
            label_list.append('b')
        elif (merchant_info.loc[i, feature] >= cd) & (merchant_info.loc[i, feature] < bc):
            label_list.append('c')
        elif (merchant_info.loc[i, feature] >= de) & (merchant_info.loc[i, feature] < cd):
            label_list.append('d')
        elif merchant_info.loc[i, feature] < de:
            label_list.append('e')

    ## mark each entry of this feature column according to the score_criteria given.
    for j in label_list:
        score_list.append(score_criteria[feature][j])

    return score_list

## Rank Algorithm

In [10]:
## Function used to rank mechant iteratively and then find the top 10 merchants to collaborate.

## merchant_info (a pd.DataFrame) is the current merchant information that are considered in our ranking system
## feature (a string) is the numeric feature we are working on
## score_criteria (a dict of dict) is the score criteria we are using for marking different features.
## remove_rate (a positive float {0,1}) is the rate to remove merchants classified as poor choice
## top_n (a positive integer) is the number of merchant to be recommended.

## Note: the lower the remove_rate, the higher the accuracy but also the longer the waiting time. (too low may cause overfitting)

def rank_algorithm(merchant_info, feature_name, score_criteria, remove_rate, top_n):
    # Stop when we find the asked number of merchants as recommendation
    while (len(merchant_info) > top_n) & (len(merchant_info) * (1-remove_rate) > top_n):
        # In each run, set the score of all merchant to zero as a fresh analysis
        merchant_info['score'] = 0

        # The main part of our rank system to mark and sum scores of all feature columns 
        for feature in feature_name:
            if feature == 'revenue_level':
                for i in range(len(merchant_info)):
                    merchant_info.loc[i, 'score'] += score_criteria[feature][merchant_info.loc[i, feature]]
            else:
                # use of mark algorthm
                merchant_info['score'] += mark_algorithm(merchant_info, feature, score_criteria)

        # sort all merchants by their total score        
        merchant_info = merchant_info.sort_values(by=['score'], ascending=False).reset_index(drop=True)
        # remove a percentage number of merchants having the lowest score (according to remove_rate given)
        merchant_info = merchant_info.drop(merchant_info.tail(int(len(merchant_info) * remove_rate)).index)
        # remove the score column after sorting and dropping merchants. (keep data integrity for next run of this algorithm)
        merchant_info = merchant_info.drop(['score'], axis=1)
        
        # if the stop criteria is not met, keep running to remove merchants that are less likely to be beneficial.
        return rank_algorithm(merchant_info, feature_name, score_criteria, remove_rate, top_n)
    
    # Show the top n merchants once stop criteria are met.
    return merchant_info.head(top_n)

In [11]:
## Run Run Run!!!!!!!!!!
top_n_merchant = rank_algorithm(merchant_info, featrue_name, score_criteria, 0.1, 100)

In [12]:
top_n_merchant.head(10)

,merchant_abn,transaction_count,field,take_rate,revenue_level,total_revenue,mean_consumer_income,fraud_count,main_business_area_popu
0,32361057556,74770,"gift, card, novelty, and souvenir shops",6.61,a,8218143.78,827.606967,18,273854
1,90087872851,19936,"digital goods: books, movies, music",6.77,a,1053355.16,856.885699,4,278870
2,45629217853,198936,"gift, card, novelty, and souvenir shops",6.98,a,7325106.13,824.636026,43,261909
3,41944909975,29998,"books, periodicals, and newspapers",0.16,e,7333635.99,823.607197,6,289380
4,52160665475,12085,"digital goods: books, movies, music",6.88,a,1062228.07,840.583128,3,466414
5,34179569263,20178,"music shops - musical instruments, pianos, and...",4.72,b,6820271.18,839.579863,8,344270
6,64403598239,98928,"music shops - musical instruments, pianos, and...",6.31,a,7726723.97,830.016924,19,225034
7,89726005175,188223,tent and awning shops,6.01,a,7764006.97,824.330699,37,350738
8,48534649627,57872,"opticians, optical goods, and eyeglasses",6.64,a,8200624.34,834.195233,14,172424
9,21772962346,29645,"florists supplies, nursery stock, and flowers",6.63,a,4503378.53,836.780330,11,115999


In [32]:
# The summary of top 100 merchants
# Revenue level count
top_n_merchant.groupby('field').count()

,merchant_abn,transaction_count,take_rate,revenue_level,total_revenue,mean_consumer_income,fraud_count,main_business_area_popu
field,,,,,,,,
"antique shops - sales, repairs, and restoration services",1,1,1,1,1,1,1,1
art dealers and galleries,1,1,1,1,1,1,1,1
artist supply and craft shops,4,4,4,4,4,4,4,4
bicycle shops - sales and service,1,1,1,1,1,1,1,1
bicycle shops - sales and service,3,3,3,3,3,3,3,3
"books, periodicals, and newspapers",5,5,5,5,5,5,5,5
"cable, satellite, and other pay television and radio services",1,1,1,1,1,1,1,1
"computer programming , data processing, and integrated systems design services",2,2,2,2,2,2,2,2
"computers, computer peripheral equipment, and software",7,7,7,7,7,7,7,7


In [30]:
# The top 100 merchants take rate mean and median
print(top_n_merchant['take_rate'].mean())
print(top_n_merchant['take_rate'].median())
print("comparison to the original merchant")
print(merchant_info['take_rate'].mean())
print(merchant_info['take_rate'].median())

5.1465000000000005
5.875
comparison to the original merchant
4.397575757575766
4.5


In [31]:
# The top 100 revenue mean and median
print(top_n_merchant['total_revenue'].mean())
print(top_n_merchant['total_revenue'].median())
print("comparison to the original merchant")
print(merchant_info['total_revenue'].mean())
print(merchant_info['total_revenue'].median())

3719929.8474999983
2953902.21
comparison to the original merchant
468184.54723050207
123231.16500000001


## Prepare the segment merchant data for ranking
### Three selected segments are "tent and awning shops", "telecom" and "shoe shops"

# Design a score criteria with high priority on take rate
## specifically for the segment jewlery

In [13]:
top_10_merchant_jewlery = rank_algorithm(merchant_jewl, featrue_name, score_criteria, 0.2, 10)

In [14]:
# This is the top 10 jewlery shops when using standard criteria
top_10_merchant_jewlery

,merchant_abn,transaction_count,field,take_rate,revenue_level,total_revenue,mean_consumer_income,fraud_count,main_business_area_popu
0,19492220327,723,"jewelry, watch, clock, and silverware shops",4.93,b,7198756.77,815.730705,128,232303
1,15043504837,175,"jewelry, watch, clock, and silverware shops",4.62,b,2912189.09,801.674285,62,237617
2,75265429612,16,"jewelry, watch, clock, and silverware shops",4.92,b,156113.21,1942.700001,4,624063
3,90918180829,493,"jewelry, watch, clock, and silverware shops",2.62,c,5072774.57,787.019473,100,242001
4,43838212570,39,"jewelry, watch, clock, and silverware shops",3.36,b,318735.49,819.292311,5,359422
5,12983555508,6,"jewelry, watch, clock, and silverware shops",5.09,b,55507.02,813.250000,1,478481
6,41331227038,34,"jewelry, watch, clock, and silverware shops",3.97,b,157573.05,810.338237,0,618091
7,23709946765,170,"jewelry, watch, clock, and silverware shops",3.59,b,1270977.51,789.755883,20,262181
8,66228393506,92,"jewelry, watch, clock, and silverware shops",1.92,c,1118797.54,993.461957,17,77875
9,44345785419,81,"jewelry, watch, clock, and silverware shops",6.07,a,1211300.90,791.981481,23,164784


### Next want to see the rank when using designed score crietria

### Buiness rule
No.1: Merchant care more about the consumption ability of their target customers (High mean income & low population > low income & large population )

No.2: The BNPL company tends to select the merchants allow them to earn more (High take rate is prefered)

No.3: Combined with No.2, the BNPL company like the merchant has frequent transactions (Large transactions count is prefered)

No.3 alternative: BNPL company may like the merchant has large total revenue 

No.4: Large penalty on fraud probability as this segment is related to luxury goods

In [15]:
score_criteria_jewlery = { \
    'transaction_count': {'a':8, 'b': 6, 'c': 4, 'd': 2, 'e': 0}, \
        'take_rate': {'a':8, 'b': 6, 'c': 4, 'd': 2, 'e': 0}, \
            'revenue_level': {'a':0, 'b': 1, 'c': 2, 'd': 3, 'e': 4}, \
                'total_revenue': {'a': 6, 'b': 4.5, 'c': 3, 'd': 1.5, 'e': 0}, \
                    'mean_consumer_income': {'a':8, 'b': 6, 'c': 4, 'd': 2, 'e': 0}, \
                        'fraud_count': {'a':0, 'b': 0, 'c': 0, 'd': 0, 'e': 0}, \
                            'main_business_area_popu': {'a':4, 'b': 3.5, 'c': 3, 'd': 2.5, 'e': 2}}

In [16]:
top_10_merchant_jewlery_designed = rank_algorithm(merchant_jewl, featrue_name, score_criteria_jewlery, 0.2, 10)

In [17]:
top_10_merchant_jewlery_designed

,merchant_abn,transaction_count,field,take_rate,revenue_level,total_revenue,mean_consumer_income,fraud_count,main_business_area_popu
0,19492220327,723,"jewelry, watch, clock, and silverware shops",4.93,b,7198756.77,815.730705,128,232303
1,15043504837,175,"jewelry, watch, clock, and silverware shops",4.62,b,2912189.09,801.674285,62,237617
2,90918180829,493,"jewelry, watch, clock, and silverware shops",2.62,c,5072774.57,787.019473,100,242001
3,23709946765,170,"jewelry, watch, clock, and silverware shops",3.59,b,1270977.51,789.755883,20,262181
4,75265429612,16,"jewelry, watch, clock, and silverware shops",4.92,b,156113.21,1942.700001,4,624063
5,66228393506,92,"jewelry, watch, clock, and silverware shops",1.92,c,1118797.54,993.461957,17,77875
6,44345785419,81,"jewelry, watch, clock, and silverware shops",6.07,a,1211300.90,791.981481,23,164784
7,10596295795,97,"jewelry, watch, clock, and silverware shops",6.84,a,1122230.91,765.761857,19,146029
8,29253286472,173,"jewelry, watch, clock, and silverware shops",3.53,b,1326580.35,783.850869,17,201176
9,57860746842,99,"jewelry, watch, clock, and silverware shops",4.64,b,1115039.24,806.882828,16,40837


## Specifically for the segment shoe shops

### First see the rank using standard score critria

In [18]:
top_10_merchant_shoe = rank_algorithm(merchant_shoe, featrue_name, score_criteria, 0.2, 10)

In [19]:
top_10_merchant_shoe

,merchant_abn,transaction_count,field,take_rate,revenue_level,total_revenue,mean_consumer_income,fraud_count,main_business_area_popu
0,93558142492,19080,shoe shops,3.41,b,7737603.79,833.856970,108,212540
1,66842618444,4616,shoe shops,6.93,a,1133102.58,805.773527,3,325060
2,17431888602,2638,shoe shops,4.27,b,1314194.35,811.043214,1,367238
3,11439466003,26038,shoe shops,5.71,a,4246363.75,822.899966,1,166304
4,96161947306,11663,shoe shops,4.52,b,1158804.32,823.588981,4,168749
5,56796971172,3168,shoe shops,5.81,a,1291325.20,843.482607,1,197524
6,19465180165,1989,shoe shops,6.73,a,251053.26,853.217094,1,308397
7,24070342853,8119,shoe shops,4.41,b,1082801.85,859.248589,1,47786
8,66678764219,3724,shoe shops,4.47,b,1222621.59,822.545113,0,287321
9,67691683330,2593,shoe shops,1.47,c,1258563.08,873.176552,0,199252


### The top 10 shoe shops when applying designed score criteria

### Business rule
No.1: Merchant care more about the numbers of potential customers (low mean income & large population > High income & small population )

No.2: The BNPL company tends to select the merchants considered to be stable (large total transactions count is prefered)

No.3 small penalty on fraud probability as shoes are usually considered to be affordable

In [20]:
score_criteria_shoe = { \
    'transaction_count': {'a':8, 'b': 6, 'c': 4, 'd': 2, 'e': 0}, \
        'take_rate': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
            'revenue_level': {'a':0, 'b': 1, 'c': 2, 'd': 3, 'e': 4}, \
                'total_revenue': {'a': 4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
                    'mean_consumer_income': {'a':4, 'b': 3.5, 'c': 3, 'd': 2.5, 'e': 2}, \
                        'fraud_count': {'a':0, 'b': 0, 'c': 0, 'd': 0, 'e': 0}, \
                            'main_business_area_popu': {'a':8, 'b': 6, 'c': 4, 'd': 2, 'e': 0}}

In [21]:
top_10_merchant_shoe_designed = rank_algorithm(merchant_shoe, featrue_name, score_criteria_shoe, 0.2, 10)

In [22]:
top_10_merchant_shoe_designed

,merchant_abn,transaction_count,field,take_rate,revenue_level,total_revenue,mean_consumer_income,fraud_count,main_business_area_popu
0,93558142492,19080,shoe shops,3.41,b,7737603.79,833.856970,108,212540
1,11439466003,26038,shoe shops,5.71,a,4246363.75,822.899966,1,166304
2,45559085309,11146,shoe shops,5.58,a,837454.32,807.527956,0,364733
3,66842618444,4616,shoe shops,6.93,a,1133102.58,805.773527,3,325060
4,60978195146,24012,shoe shops,1.44,c,1198451.79,822.530447,6,163332
5,79269463423,5709,shoe shops,6.44,a,942558.50,808.204309,1,282676
6,12034469787,4661,shoe shops,5.83,a,352413.38,812.087621,2,481339
7,96161947306,11663,shoe shops,4.52,b,1158804.32,823.588981,4,168749
8,91630014920,2540,shoe shops,5.51,a,827779.48,830.902047,0,368384
9,17431888602,2638,shoe shops,4.27,b,1314194.35,811.043214,1,367238


## Specifically for the segment tent shops

Standard score criteria

In [23]:
top_10_merchant_tents = rank_algorithm(merchant_tents, featrue_name, score_criteria, 0.2, 10)

In [24]:
top_10_merchant_tents

,merchant_abn,transaction_count,field,take_rate,revenue_level,total_revenue,mean_consumer_income,fraud_count,main_business_area_popu
0,89726005175,188223,tent and awning shops,6.01,a,7764006.97,824.330699,37,350738
1,38700038932,6244,tent and awning shops,6.31,a,8350160.13,841.472757,0,68076
2,46331355995,1728,tent and awning shops,6.27,a,769239.96,849.339236,1,374766
3,64203420245,227504,tent and awning shops,2.86,c,6584186.21,828.833363,37,261909
4,80551528183,6813,tent and awning shops,4.81,b,7673689.76,815.279069,3,355759
5,57900494384,58472,tent and awning shops,6.39,a,1210016.36,829.623753,10,229065
6,90568944804,9327,tent and awning shops,4.10,b,8377145.54,832.787991,87,285945
7,59737558413,1330,tent and awning shops,6.12,a,904208.82,860.836466,0,405736
8,49891706470,215786,tent and awning shops,5.80,a,6254617.02,829.047491,49,237842
9,78080443264,941,tent and awning shops,0.27,e,19082.83,887.658980,12,325562


### Designed score criteria


### Business rule

No.1: The BNPL company tends to select the merchants considered to be stable (large total transactions count and large total revenue is prefered)

No.2 large penalty on fraud probability as clients dont like the risks

In [25]:
# Higher priority means the algorithm tends to select the merchant that can make the BNPL company earn more on each transaction
score_criteria_tents = { \
    'transaction_count': {'a':8, 'b': 6, 'c': 4, 'd': 2, 'e': 0}, \
        'take_rate': {'a':40, 'b': 30, 'c': 20, 'd': 10, 'e': 0}, \
            'revenue_level': {'a':0, 'b': 1, 'c': 2, 'd': 3, 'e': 4}, \
                'total_revenue': {'a': 8, 'b': 6, 'c': 4, 'd': 2, 'e': 0}, \
                    'mean_consumer_income': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}, \
                        'fraud_count': {'a':0, 'b': 0, 'c': 0, 'd': 0, 'e': 0}, \
                            'main_business_area_popu': {'a':4, 'b': 3, 'c': 2, 'd': 1, 'e': 0}}

In [26]:
top_10_merchant_tents_designed = rank_algorithm(merchant_tents, featrue_name, score_criteria_tents, 0.2, 10)

In [27]:
top_10_merchant_tents_designed

,merchant_abn,transaction_count,field,take_rate,revenue_level,total_revenue,mean_consumer_income,fraud_count,main_business_area_popu
0,70610974780,13995,tent and awning shops,6.93,a,577739.46,807.230303,2,109788
1,22644977942,195,tent and awning shops,6.96,a,175906.12,789.087692,0,192876
2,96911331900,383,tent and awning shops,6.88,a,433536.95,778.125848,0,212840
3,34703763702,22,tent and awning shops,6.98,a,23989.62,769.054543,0,65295
4,66432686430,1650,tent and awning shops,6.81,a,1123207.41,872.342182,0,131294
5,51221851716,419,tent and awning shops,6.77,a,11955.38,844.345586,0,229357
6,27573216953,40,tent and awning shops,6.82,a,15127.18,788.825000,0,91412
7,63523606379,305,tent and awning shops,6.80,a,199957.24,783.138688,0,114468
8,64203420245,227504,tent and awning shops,2.86,c,6584186.21,828.833363,37,261909
9,57900494384,58472,tent and awning shops,6.39,a,1210016.36,829.623753,10,229065
